In [ ]:
import os
import pandas as pd
import numpy as np
import glob

In [ ]:
base_dir = "C:\\Users\\ASABUNCUOGLU13\\Documents\\data\\vol08\\"
input_dir = "C:\\Users\\ASABUNCUOGLU13\\Documents\\data\\vol08\\group"
scores_dir = "..\\scores\\vol08\\group\\levels"
out_dir = "C:\\Users\\ASABUNCUOGLU13\\Documents\\data\\vol08\\engagement-slices"
video_name = "{}.mp4"
csv_name = "list.csv"

engagement_levels = ["-2", "-1", "0", "1", "2"]

In [ ]:
!mkdir {out_dir}
for e in engagement_levels:
    !mkdir {os.path.join(out_dir, e)}

In [ ]:
def sec_convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%02d:%02d:%02d" % (hour, minutes, seconds)

In [ ]:
video_list = []

In [ ]:
v = 0
for t in range(len(glob.glob1(input_dir,"*.mp4"))):
    print(t)
    df = pd.read_csv(os.path.join(scores_dir, "v%d.csv" % t), sep=',')
    df.head()

    start_time = 0
    i = 10
    slices = []

    for curr_level in df['group']:
        next_level = df['group'][i]
            
        if(curr_level != next_level):
            #print(start_time, curr_level)
            slices.append([sec_convert(start_time), sec_convert(i), curr_level])
            curr_level = next_level
            start_time = i

        if i < len(df['group'])-1:
            i += 2

    for slice in slices:
        !ffmpeg -ss {slice[0]} -to {slice[1]} -i {input_dir + "\\v" + video_name.format(str(t))} -c copy {out_dir + "\\" + str(slice[2]) + "\\" + str(v)}.mp4 -hide_banner -loglevel error
        !ffmpeg -i {out_dir + "\\" + str(slice[2]) + "\\" + str(v)}.mp4 -vn -acodec pcm_s16le -ar 44100 -ac 1 -af "afftdn=nf=-25" {out_dir + "\\" + str(slice[2]) + "\\" + str(v)}.wav -hide_banner -loglevel error
        video_list.append([str(v) + '.mp4', slice[2]])
        v += 1

In [ ]:
df = pd.DataFrame (video_list, columns = ['video_name', 'tag'])

In [ ]:
df['tag'] = df['tag'].map({-2: 'HD', -1:'D', 0:'M', 1: 'E', 2: 'HE'})

In [ ]:
df.to_csv(os.path.join(out_dir, csv_name), index=False)